# Strategy

## O que é?

A ideia do padrao _Strategy_ é definir uma família de métodos, encapsulá-los e permitir que eles sejam intercambiáveis.

## Exemplo

Um exemplo clássico de uso do padrao Strategy seria a criacao de personagens num jogo no estilo RPG. Suponhamos que o jogo tenha personagens com as seguintes características: 

In [4]:
class Character:
    
    def __init__(self, name: str, job: str):
        self.name = name
        self.job = job
    
    def attack(self):
        print(f"{self.name} is killing you")


c1 = Character('Player1', 'Fighter')
c2 = Character('Player2', 'Black Mage')
c3 = Character('Player3', 'Bard')

c1.attack()
c2.attack()
c3.attack()

Player1 is killing you
Player2 is killing you
Player3 is killing you


Vamos supor que queremos implementar métodos de ataque distintos para cada job. Uma implementacao naive seria a seguinte:

In [6]:
class Character:
    def __init__(self, name: str, job: str):
        self.name = name
        self.job = job
    
    def attack(self):
        if self.job == 'Fighter':
            print(f"{self.name} is killing you with his mighty fists")
        if self.job == 'Black Mage':
            print(f"{self.name} is killing you with the power of magic")
        if self.job == 'Bard':
            print(f"{self.name} is killing you with the power of music")
            
c1 = Character('Player1', 'Fighter')
c2 = Character('Player2', 'Black Mage')
c3 = Character('Player3', 'Bard')

c1.attack()
c2.attack()
c3.attack()

Player1 is killing you with his mighty fists
Player2 is killing you with the power of magic
Player3 is killing you with the power of music


É fácil ver como a complexidade do método `attack` pode explodir com a criacao de novos jobs ou métodos de ataque. Uma maneira otimista de contornar isso seria usando inheritance e criando subclasses para cada job

In [3]:
import abc


class Character(abc.ABC):
    
    def __init__(self, name: str, job: str):
        self.name = name
        self.job = job
    
    @abc.abstractmethod
    def attack(self):
        pass

class Fighter(Character):

    def __init__(self, name: str):
        super().__init__(name, "Fighter")
    
    def attack(self):
        print(f"{self.name} is killing you with his mighty fists")

class BlackMage(Character):
    
    def __init__(self, name: str):
        super().__init__(name, "Black Mage")
    
    def attack(self):
        print(f"{self.name} is killing you with the power of magic")

class Bard(Character):
    
    def __init__(self, name: str):
        super().__init__(name, "Bard")
    
    def attack(self):
        print(f"{self.name} is killing you with the power of music")

c1 = Fighter('Player1')
c2 = BlackMage('Player2')
c3 = Bard('Player3')

c1.attack()
c2.attack()
c3.attack()

Player1 is killing you with his mighty fists
Player2 is killing you with the power of magic
Player3 is killing you with the power of music


O uso de subclasses evita a complexidade do método `attack`, mas cria novos problemas: se desejássemos introduzir _power-ups_ ou armas para mudar o método `attack`, teríamos que reestruturar todo o código. O mesmo aconteceria se quiséssemos permitir que os personagens mudassem de job ao longo do jogo.

## Usando o padrao Strategy

In [15]:
import abc


class AttackStrategy(abc.ABC):
    
    def __init__(self, character: Character):
        self.character = character
        
    def attack(self):
        pass

    
class FistAttackStrategy(AttackStrategy):
    
    def attack(self):
        print(f"{self.character.name} is killing you with his mighty fists")
        
        
class MagicAttackStrategy(AttackStrategy):
    
    def attack(self):
        print(f"{self.character.name} is killing you with the power of magic")  

        
class MusicAttackStrategy(AttackStrategy):
    
    def attack(self):
        print(f"{self.character.name} is killing you with the power of music")         
        
        
class Character(abc.ABC):
    
    def __init__(self, name: str, job: str):
        self.name = name
        self.job = job
        self.attack_strategy = AttackStrategy(self)
    
    def attack(self):
        self.attack_strategy.attack()
         

class Fighter(Character):

    def __init__(self, name: str):
        super().__init__(name, "Fighter")
        self.attack_strategy = FistAttackStrategy(self)
        

class BlackMage(Character):
    
    def __init__(self, name: str):
        super().__init__(name, "Black Mage")
        self.attack_strategy = MagicAttackStrategy(self)

        
class Bard(Character):
    
    def __init__(self, name: str):
        super().__init__(name, "Bard")
        self.attack_strategy = MusicAttackStrategy(self)
    
    

c1 = Fighter('Player1')
c2 = BlackMage('Player2')
c3 = Bard('Player3')

c1.attack()
c2.attack()
c3.attack()


Player1 is killing you with his mighty fists
Player2 is killing you with the power of magic
Player3 is killing you with the power of music


A primeira vista, estamos usando muito mais código para atingir o mesmo resultado. No entanto, o uso do _strategy pattern_ nos permite ter muito mais flexibilidade. Se o nosso _fighter_ decidir estudar magia negra nas horas vagas, por exemplo, nosso jogo permite isso sem problemas.

In [19]:
c1.attack_strategy = MagicAttackStrategy(c1)
c1.attack()

Player1 is killing you with the power of magic


Se desejamos criar um power-up ̣_fireball_, por exemplo, só precisamos adicionar uma nova classe `AttackStrategy`. Nao é necessário fazer qualquer mudanca à classe `Character`.

In [20]:
class FireballAttackStrategy(AttackStrategy):

    def attack(self):
        print(f"{self.character.name} is killing you with FIRE")

        
c2.attack_strategy = FireballAttackStrategy(c2)
c2.attack()
    

Player2 is killing you with FIRE


## Vantagens e desvantagens

### Pros:
- Aumenta a flexibilidade 
- Reduz a complexidade da classe que implementa o método
- Desacopla o código de uma classe específica

### Cons:
- Força a criação de novas classes
- Adiciona complexidade à criacao de objeto

## Estrutura

![struct](https://www.dofactory.com/images/diagrams/net/strategy.gif)

## Detalhes de implementacao

### Acessando a informacoes do contexto

Para que o padrao _Strategy_ funcione, é necessário que o objeto Strategy tenha acesso às propriedades do cliente que o implementa. O livro _Design Patterns_ descreve duas maneiras de solucionar esse problema:

1 - Passar os detalhes de cliente como parâmetros para os métodos do objeto Strategy. Isso reduz a interdependência entre a classe Strategy e o cliente, mas dá ao cliente a responsabilidade adicional de passar a informacao ao método Strategy.

2 - Passar o cliente em si como parâmetro para o objeto Strategy dar à classe Strategy a responsabilidade de extrair a informacao necessária. Essa foi a opcao adotada no exemplo acima. O benefício dessa solucao é que ela reduz as responsabilidades do cliente, embora aumente a interdependencia entre a classe Strategy e o cliente.

### Uso em conjunto com _Factory Method_

Por aumentar a complexidade da criacao dos objetos que o implementam, o padrao _Strategy_ é um bom candidato para uso em conjunto com o _Factory Method_